# Testing the Machine Learning Model

I will keep this notebook updated in a way that it can always be used to test the newest version of the NLP model for classifying AI headlines.

For now, you can replace the sentence in the `predict` function call at the last line to get a prediction for whatever sentence you add.

TODO: add option for a graphical interface with Gradio

TODO: Check our [multi-label case](https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_multi_label_classification.ipynb)

[this](https://github.com/Dirkster99/PyNotes/blob/master/Transformers/LocalModelUsage_Finetuning/30%20MultiClass%20Classification%20in%2010%20Minutes%20with%20BERT-TensorFlow-SoftMax-LocalModel.ipynb) may help too?

In [8]:
# Updates the transformers library to the latest version. WARNING: If you don't have it installed, it's a couple GBs of data.
!pip install -Uqq transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
from transformers import TextClassificationPipeline
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification

import json

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
names = {}

with open('data/data.json') as f:
    d = json.load(f)
    for i, name in enumerate(d.keys()):
        names[f"LABEL_{i}"] = name
            
names

{'LABEL_0': 'agency',
 'LABEL_1': 'humanComparison',
 'LABEL_2': 'hyperbole',
 'LABEL_3': 'historyComparison',
 'LABEL_4': 'unjustClaims',
 'LABEL_5': 'deepSounding',
 'LABEL_6': 'skeptics',
 'LABEL_7': 'deEmphasize',
 'LABEL_8': 'performanceNumber',
 'LABEL_9': 'inscrutable',
 'LABEL_10': 'objective'}

In [4]:
model = BertForSequenceClassification.from_pretrained('test_trainer/checkpoint-500/',local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

/home/xt0r3/.local/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [5]:
def predict(text):
    preds = pipe(text)[0]
    preds = list(map(lambda pred: {"label": names[pred["label"]], "score": pred["score"]}, preds))
    print(preds)

In [10]:
# Predict scores
predict("Machine Learning is at the forefront of education, replacing human jobs")
# Note that it does a weird prediction, saying that this is a deep-sounding headline, even though it also attributes agency, compares it with humans, uses the hyperbole and 
# has unjust claims in the headline. 
# TODO: Find a model suitable for multiple classification?
# Check out https://discuss.huggingface.co/t/fine-tune-for-multiclass-or-multilabel-multiclass/4035
# Even better: https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_multi_label_classification.ipynb

[{'label': 'agency', 'score': 0.0001564613776281476}, {'label': 'humanComparison', 'score': 0.0009414521045982838}, {'label': 'hyperbole', 'score': 0.0003454808611422777}, {'label': 'historyComparison', 'score': 0.0010318453423678875}, {'label': 'unjustClaims', 'score': 0.0004964807303622365}, {'label': 'deepSounding', 'score': 0.99512779712677}, {'label': 'skeptics', 'score': 0.0005015040514990687}, {'label': 'deEmphasize', 'score': 8.166562474798411e-05}, {'label': 'performanceNumber', 'score': 0.00028639606898650527}, {'label': 'inscrutable', 'score': 0.00028850772650912404}, {'label': 'objective', 'score': 0.0007424494251608849}]
